In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import plotly.express as px
from jupyter_dash import JupyterDash

### Importing Data

#### The link for data (https://www.kaggle.com/rounakbanik/the-movies-dataset?select=ratings.csv). Data is same that is provided in Data Scientist Task Details

In [2]:
pd.set_option("display.max_columns", None)
df_ratings = pd.read_csv("ratings.csv",low_memory=False)
df_links = pd.read_csv("links.csv",low_memory=False)
df_keywords = pd.read_csv("keywords.csv",low_memory=False)
df_movies = pd.read_csv("movies_metadata.csv",low_memory=False)
df_credits = pd.read_csv("credits.csv",low_memory=False)

### Preprocessing and cleaning

In [3]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
df_ratings = df_ratings.groupby(df_ratings.columns.to_list(),as_index=False).size()
df_ratings

,userId,movieId,rating,timestamp,size
0,1,110,1.0,1425941529,1
1,1,147,4.5,1425942435,1
2,1,858,5.0,1425941523,1
3,1,1221,5.0,1425941546,1
4,1,1246,5.0,1425941556,1
...,...,...,...,...,...
26024284,270896,58559,5.0,1257031564,1
26024285,270896,60069,5.0,1257032032,1
26024286,270896,63082,4.5,1257031764,1
26024287,270896,64957,4.5,1257033990,1


In [5]:
df_ratings["timestamp"] = df_ratings["timestamp"].apply(date.fromtimestamp)

In [6]:
df_ratings.head()

,userId,movieId,rating,timestamp,size
0,1,110,1.0,2015-03-10,1
1,1,147,4.5,2015-03-10,1
2,1,858,5.0,2015-03-10,1
3,1,1221,5.0,2015-03-10,1
4,1,1246,5.0,2015-03-10,1


In [7]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
 4   size       int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 992.7+ MB


In [8]:
df_ratings["timestamp"] = pd.to_datetime(df_ratings["timestamp"])

In [9]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   movieId    int64         
 2   rating     float64       
 3   timestamp  datetime64[ns]
 4   size       int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 992.7 MB


In [10]:
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [11]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


In [12]:
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [13]:
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [14]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [15]:
df_movies.rename(columns={"id":"tmdbId","imdb_id":"imdbId"}, inplace=True)
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,imdbId,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   tmdbId                 45466 non-null  object 
 6   imdbId                 45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [17]:
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'tmdbId', 'imdbId', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [18]:
df_movies = df_movies[["title","tmdbId","release_date"]]

In [19]:
df_movies.head()

,title,tmdbId,release_date
0,Toy Story,862,1995-10-30
1,Jumanji,8844,1995-12-15
2,Grumpier Old Men,15602,1995-12-22
3,Waiting to Exhale,31357,1995-12-22
4,Father of the Bride Part II,11862,1995-02-10


In [20]:
df_movies.isna().sum()

title            6
tmdbId           0
release_date    87
dtype: int64

In [21]:
df_movies.dropna(inplace=True)
df_movies.head()

,title,tmdbId,release_date
0,Toy Story,862,1995-10-30
1,Jumanji,8844,1995-12-15
2,Grumpier Old Men,15602,1995-12-22
3,Waiting to Exhale,31357,1995-12-22
4,Father of the Bride Part II,11862,1995-02-10


In [22]:
df_movies.isna().sum()

title           0
tmdbId          0
release_date    0
dtype: int64

In [23]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         45376 non-null  object
 1   tmdbId        45376 non-null  object
 2   release_date  45376 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [24]:
df_movies["release_date"] = pd.to_datetime(df_movies["release_date"])
df_movies.head()

,title,tmdbId,release_date
0,Toy Story,862,1995-10-30
1,Jumanji,8844,1995-12-15
2,Grumpier Old Men,15602,1995-12-22
3,Waiting to Exhale,31357,1995-12-22
4,Father of the Bride Part II,11862,1995-02-10


In [25]:
df_movies["tmdbId"] = pd.to_numeric(df_movies["tmdbId"])

In [26]:
df_movies.head()

,title,tmdbId,release_date
0,Toy Story,862,1995-10-30
1,Jumanji,8844,1995-12-15
2,Grumpier Old Men,15602,1995-12-22
3,Waiting to Exhale,31357,1995-12-22
4,Father of the Bride Part II,11862,1995-02-10


In [27]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         45376 non-null  object        
 1   tmdbId        45376 non-null  int64         
 2   release_date  45376 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.4+ MB


In [28]:
df = pd.merge(df_ratings, df_links, on="movieId")
df = pd.merge(df, df_movies, on="tmdbId")
df.head()

,userId,movieId,rating,timestamp,size,imdbId,tmdbId,title,release_date
0,1,110,1.0,2015-03-10,1,112573,197.0,Braveheart,1995-05-24
1,11,110,3.5,2009-01-11,1,112573,197.0,Braveheart,1995-05-24
2,22,110,5.0,2005-03-27,1,112573,197.0,Braveheart,1995-05-24
3,24,110,5.0,2001-01-19,1,112573,197.0,Braveheart,1995-05-24
4,29,110,3.0,2003-01-31,1,112573,197.0,Braveheart,1995-05-24


In [29]:
df.drop(columns=["imdbId","tmdbId"], inplace=True)
df.head()

,userId,movieId,rating,timestamp,size,title,release_date
0,1,110,1.0,2015-03-10,1,Braveheart,1995-05-24
1,11,110,3.5,2009-01-11,1,Braveheart,1995-05-24
2,22,110,5.0,2005-03-27,1,Braveheart,1995-05-24
3,24,110,5.0,2001-01-19,1,Braveheart,1995-05-24
4,29,110,3.0,2003-01-31,1,Braveheart,1995-05-24


In [30]:
df.isnull().sum()

userId          0
movieId         0
rating          0
timestamp       0
size            0
title           0
release_date    0
dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25990591 entries, 0 to 25990590
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   userId        int64         
 1   movieId       int64         
 2   rating        float64       
 3   timestamp     datetime64[ns]
 4   size          int64         
 5   title         object        
 6   release_date  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 1.5+ GB


In [32]:
df.head()

,userId,movieId,rating,timestamp,size,title,release_date
0,1,110,1.0,2015-03-10,1,Braveheart,1995-05-24
1,11,110,3.5,2009-01-11,1,Braveheart,1995-05-24
2,22,110,5.0,2005-03-27,1,Braveheart,1995-05-24
3,24,110,5.0,2001-01-19,1,Braveheart,1995-05-24
4,29,110,3.0,2003-01-31,1,Braveheart,1995-05-24


In [33]:
df['date_difference'] = (df['timestamp'] - df['release_date']).dt.days

In [34]:
df.head()

,userId,movieId,rating,timestamp,size,title,release_date,date_difference
0,1,110,1.0,2015-03-10,1,Braveheart,1995-05-24,7230
1,11,110,3.5,2009-01-11,1,Braveheart,1995-05-24,4981
2,22,110,5.0,2005-03-27,1,Braveheart,1995-05-24,3595
3,24,110,5.0,2001-01-19,1,Braveheart,1995-05-24,2067
4,29,110,3.0,2003-01-31,1,Braveheart,1995-05-24,2809


In [35]:
df['date_difference_days'] = (df['timestamp'] - df['release_date'])

In [36]:
df

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
0,1,110,1.0,2015-03-10,1,Braveheart,1995-05-24,7230,7230 days
1,11,110,3.5,2009-01-11,1,Braveheart,1995-05-24,4981,4981 days
2,22,110,5.0,2005-03-27,1,Braveheart,1995-05-24,3595,3595 days
3,24,110,5.0,2001-01-19,1,Braveheart,1995-05-24,2067,2067 days
4,29,110,3.0,2003-01-31,1,Braveheart,1995-05-24,2809,2809 days
...,...,...,...,...,...,...,...,...,...
25990586,270887,159050,5.0,2016-09-01,1,The Pilot's Wife,2005-08-17,4033,4033 days
25990587,270887,159053,5.0,2017-02-13,1,In Her Line of Fire,2006-04-21,3951,3951 days
25990588,270887,165649,4.0,2016-11-14,1,Dead on Campus,2014-11-10,735,735 days
25990589,270887,171051,4.0,2017-04-25,1,Without Evidence,1995-10-12,7866,7866 days


In [37]:
df.sort_values(by=["date_difference"])

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
25759731,96962,1115,1.0,1996-10-27,1,The Sleepover,2013-10-12,-6194,-6194 days
25759748,169424,1115,3.0,1996-10-28,1,The Sleepover,2013-10-12,-6193,-6193 days
25759734,107513,1115,3.0,1996-10-30,1,The Sleepover,2013-10-12,-6191,-6191 days
25759753,197155,1115,4.0,1996-11-03,1,The Sleepover,2013-10-12,-6187,-6187 days
25759746,159345,1115,2.0,1996-11-04,1,The Sleepover,2013-10-12,-6186,-6186 days
...,...,...,...,...,...,...,...,...,...
25905539,148093,148054,4.0,2016-08-26,1,Passage of Venus,1874-12-09,51760,51760 days
25905537,74311,148054,3.0,2016-11-07,1,Passage of Venus,1874-12-09,51833,51833 days
25905538,139317,148054,2.0,2016-12-22,1,Passage of Venus,1874-12-09,51878,51878 days
25905540,165352,148054,0.5,2016-12-29,1,Passage of Venus,1874-12-09,51885,51885 days


In [38]:
df.sort_values(by=["date_difference"],ascending=False)

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
25905536,68731,148054,1.0,2017-02-20,1,Passage of Venus,1874-12-09,51938,51938 days
25905540,165352,148054,0.5,2016-12-29,1,Passage of Venus,1874-12-09,51885,51885 days
25905538,139317,148054,2.0,2016-12-22,1,Passage of Venus,1874-12-09,51878,51878 days
25905537,74311,148054,3.0,2016-11-07,1,Passage of Venus,1874-12-09,51833,51833 days
25905539,148093,148054,4.0,2016-08-26,1,Passage of Venus,1874-12-09,51760,51760 days
...,...,...,...,...,...,...,...,...,...
25759746,159345,1115,2.0,1996-11-04,1,The Sleepover,2013-10-12,-6186,-6186 days
25759753,197155,1115,4.0,1996-11-03,1,The Sleepover,2013-10-12,-6187,-6187 days
25759734,107513,1115,3.0,1996-10-30,1,The Sleepover,2013-10-12,-6191,-6191 days
25759748,169424,1115,3.0,1996-10-28,1,The Sleepover,2013-10-12,-6193,-6193 days


In [39]:
df = df.drop(df[df["date_difference"]<0].index)
df

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
0,1,110,1.0,2015-03-10,1,Braveheart,1995-05-24,7230,7230 days
1,11,110,3.5,2009-01-11,1,Braveheart,1995-05-24,4981,4981 days
2,22,110,5.0,2005-03-27,1,Braveheart,1995-05-24,3595,3595 days
3,24,110,5.0,2001-01-19,1,Braveheart,1995-05-24,2067,2067 days
4,29,110,3.0,2003-01-31,1,Braveheart,1995-05-24,2809,2809 days
...,...,...,...,...,...,...,...,...,...
25990586,270887,159050,5.0,2016-09-01,1,The Pilot's Wife,2005-08-17,4033,4033 days
25990587,270887,159053,5.0,2017-02-13,1,In Her Line of Fire,2006-04-21,3951,3951 days
25990588,270887,165649,4.0,2016-11-14,1,Dead on Campus,2014-11-10,735,735 days
25990589,270887,171051,4.0,2017-04-25,1,Without Evidence,1995-10-12,7866,7866 days


In [40]:
df.sort_values(by=["date_difference"])

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
20773348,214403,54276,4.5,2007-07-25,1,No Reservations,2007-07-25,0,0 days
25551755,143595,142975,0.5,2015-10-08,1,Navy Seals vs. Zombies,2015-10-08,0,0 days
23312490,79642,158813,2.5,2016-05-25,1,Alice Through the Looking Glass,2016-05-25,0,0 days
25919267,251652,173307,3.0,2017-07-21,1,The Gracefield Incident,2017-07-21,0,0 days
25488458,244536,144716,5.0,2015-10-22,1,Rock the Kasbah,2015-10-22,0,0 days
...,...,...,...,...,...,...,...,...,...
25905539,148093,148054,4.0,2016-08-26,1,Passage of Venus,1874-12-09,51760,51760 days
25905537,74311,148054,3.0,2016-11-07,1,Passage of Venus,1874-12-09,51833,51833 days
25905538,139317,148054,2.0,2016-12-22,1,Passage of Venus,1874-12-09,51878,51878 days
25905540,165352,148054,0.5,2016-12-29,1,Passage of Venus,1874-12-09,51885,51885 days


In [41]:
df.sort_values(by=["date_difference"],ascending=False)

,userId,movieId,rating,timestamp,size,title,release_date,date_difference,date_difference_days
25905536,68731,148054,1.0,2017-02-20,1,Passage of Venus,1874-12-09,51938,51938 days
25905540,165352,148054,0.5,2016-12-29,1,Passage of Venus,1874-12-09,51885,51885 days
25905538,139317,148054,2.0,2016-12-22,1,Passage of Venus,1874-12-09,51878,51878 days
25905537,74311,148054,3.0,2016-11-07,1,Passage of Venus,1874-12-09,51833,51833 days
25905539,148093,148054,4.0,2016-08-26,1,Passage of Venus,1874-12-09,51760,51760 days
...,...,...,...,...,...,...,...,...,...
24264051,210002,1520,4.0,1997-05-02,1,Commandments,1997-05-02,0,0 days
24869812,217873,747,5.0,1996-08-30,1,The Stupids,1996-08-30,0,0 days
12623334,149008,2006,5.0,1998-07-16,1,The Mask of Zorro,1998-07-16,0,0 days
25660068,104322,144632,2.0,2015-10-20,1,Back in Time,2015-10-20,0,0 days


In [42]:
df.drop(columns=["userId","movieId","release_date","date_difference","date_difference_days"], inplace=True)
df

,rating,timestamp,size,title
0,1.0,2015-03-10,1,Braveheart
1,3.5,2009-01-11,1,Braveheart
2,5.0,2005-03-27,1,Braveheart
3,5.0,2001-01-19,1,Braveheart
4,3.0,2003-01-31,1,Braveheart
...,...,...,...,...
25990586,5.0,2016-09-01,1,The Pilot's Wife
25990587,5.0,2017-02-13,1,In Her Line of Fire
25990588,4.0,2016-11-14,1,Dead on Campus
25990589,4.0,2017-04-25,1,Without Evidence


In [43]:
df

,rating,timestamp,size,title
0,1.0,2015-03-10,1,Braveheart
1,3.5,2009-01-11,1,Braveheart
2,5.0,2005-03-27,1,Braveheart
3,5.0,2001-01-19,1,Braveheart
4,3.0,2003-01-31,1,Braveheart
...,...,...,...,...
25990586,5.0,2016-09-01,1,The Pilot's Wife
25990587,5.0,2017-02-13,1,In Her Line of Fire
25990588,4.0,2016-11-14,1,Dead on Campus
25990589,4.0,2017-04-25,1,Without Evidence


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25980654 entries, 0 to 25990590
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   rating     float64       
 1   timestamp  datetime64[ns]
 2   size       int64         
 3   title      object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 991.1+ MB


# Visualisation

#### Visualisation using Plotly Dash 

In [45]:
app = JupyterDash(__name__)

In [46]:
app.layout = html.Div(id="parent", children=[
    
    html.Label("Number of ratings for a Movie in last 7 days to last 7 months (ref. date is last rating/comment date) :",
               style={"fontSize":20, "textAlign":"center"}),
    
    dcc.Dropdown(id="movies-dropdown-bar", 
                 options=[{"label":c, "value":c} for c in df["title"].unique()],
                 value="The Space Between Us",
                 placeholder="Select Movie",),
    
    dcc.DatePickerRange(id="date-picker-movie",
                        min_date_allowed= pd.to_datetime(df["timestamp"]).max() + relativedelta(months=-7),
                        max_date_allowed= pd.to_datetime(df["timestamp"]).max(),
                        start_date=pd.to_datetime(df["timestamp"]).max() + relativedelta(months=-3),
                        end_date=pd.to_datetime(df["timestamp"]).max(),
                        minimum_nights=6,
                        updatemode="bothdates",
                        display_format='Do MMM, YY',
                        clearable=True,
                        number_of_months_shown = 3,
                        with_portal=True,
                        ),

    html.Div([dcc.Graph(id="bar-chart-movie")]),
              
    html.Br(),          
   
    
    
    html.Label("Number of ratings for a Movie :", style={"fontSize":20, "textAlign":"center"}),
    
    dcc.Dropdown(id="movies-dropdown-pie",
                 options=[{"label":c, "value":c} for c in df["title"].unique()],
                 value="The Space Between Us",
                 placeholder="Select Movie"),
    
    html.Div([dcc.Graph(id="pie-chart")]),
    
    html.Br(), 
    
    
    
    
    html.Label("Ratings for top 10 Movie in last 7 days to last 7 months (ref. date is last rating/comment date) :",
               style={"fontSize":20, "textAlign":"center"}),
    
    dcc.DatePickerRange(id="date-picker",
                        min_date_allowed=pd.to_datetime(df["timestamp"]).max() + relativedelta(months=-7),
                        max_date_allowed=pd.to_datetime(df["timestamp"]).max(),
                        start_date=pd.to_datetime(df["timestamp"]).max() + relativedelta(days=-7),
                        end_date=pd.to_datetime(df["timestamp"]).max(),
                        minimum_nights=6,
                        updatemode="bothdates",
                        display_format='Do MMM, YY',
                        clearable=True,
                        number_of_months_shown = 3,
                        with_portal=True,
                        ),

    html.Div([dcc.Graph(id="bar-chart")
             ])
    ])

In [47]:
@app.callback(Output("bar-chart-movie", "figure"),
              [Input("movies-dropdown-bar", "value"),
               Input("date-picker-movie", "start_date"),
               Input("date-picker-movie", "end_date")])

def graph_update(movies_dropdown, start_date, end_date):
    df1 = df.loc[(df["title"]==movies_dropdown)]
    df2 = df1.drop(df1[(df1["timestamp"])<start_date].index)
    df3 = df2.drop(df2[(df2["timestamp"])>end_date].index)
    df3 = df3.groupby(['title', 'rating'], as_index=False).sum()
    fig_bar=px.bar(data_frame=df3,x="rating", y="size", color="rating")
    return(fig_bar)


@app.callback(Output("pie-chart", "figure"),
              [Input("movies-dropdown-pie", "value")])

def graph_update(movies_dropdown):
    df1 = df.loc[(df["title"]==movies_dropdown)].groupby(['title', 'rating'], as_index=False).sum()
    fig_pie=px.pie(data_frame=df1,values="size", labels="rating", hole=0.5, names="rating")
    return(fig_pie)


@app.callback(Output("bar-chart", "figure"),
              [Input("date-picker", "start_date"),
               Input("date-picker", "end_date")])

def graph_update(start_date, end_date):
    df_top_10 = df.drop(df[df["rating"]<5].index)
    df_movie_top_10 = df_top_10.drop(df_top_10[(df["timestamp"])<start_date].index)
    df_movie_top_10 = df_movie_top_10.drop(df_movie_top_10[(df_movie_top_10["timestamp"])>end_date].index)
    df_movie_top_10 = df_movie_top_10.groupby(["title","rating"],as_index=False).sum()
    df_movie_top_10.sort_values(by="size", ascending=False,inplace=True)
    fig=px.bar(data_frame=df_movie_top_10[0:10], x="title", y="size")
    return(fig)

In [48]:
app.run_server("inline")

# To open in web browser, copy and paste this link in web browser--->     http://127.0.0.1:8050/